This notebook is used to parse crsp data that only includes stocks that are ordinary/common shares and listed on the NYSE, AMEX, or NASDAQ. Also, we set negative prices equal to 'NA' since a negative price represents the midpoint of the bid-ask spread and we only want closing price values.

# CRSP Data

In [2]:
import pandas as pd
from pathlib import Path  
import datetime
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings('ignore')

In [ ]:
cd

In [6]:
cd Desktop/Caltech/Senior/Spring 2023/BEM 114

/Users/helensiavelis/Desktop/Caltech/Senior/Spring 2023/BEM 114


In [ ]:
data = pd.read_csv("crsp_1926_2020.csv")
data.date = pd.to_datetime(data.date)

## Exchange Encodings 

1 : NYSE

2 : AMEX (NYSE US/MKT)

3 : NASDAQ

4 : Arca

5 : Bats (As Quoted By NASDAQ)

6 : ?

31 : When-Issued Trading on NYSE

32 : When-Issued Trading on NYSE MKT

33 : When-ISsued Trading on Arca

## Share Encodings 

10 : US Common Stock

11 : US Common Stock

12 : Non-US Common Stock

14 : US Closed-End Funds and Unit Investment Trusts Common Stock

15 : Non-US Closed-End Funds and Unit Investment Trusts Common Stock

18 : REITs Common Stock

20 : US Certificates

21 : US Certificates

23 : US Certificates (Americus Trust Components )

24 : US Closed-End Funds and Unit Investment US Certificates

30 : Americus Trust Stocks 

31 : Americus Trust ADRs

40 : US SBIs 

41 : US SBIs

42 : Non-US SBIs

44 : US Closed-End Funds and Unit Investment SBIs

48 : REIT SBIs

70 : US Units (Depositary Units, Units Of Beneficial Interest, Units Of Limited Partnership Interest, Depositary Receipts, etc.)

71 : US Units (Depositary Units, Units Of Beneficial Interest, Units Of Limited Partnership Interest, Depositary Receipts, etc.)

72 : Non-US Units (Depositary Units, Units Of Beneficial Interest, Units Of Limited Partnership Interest, Depositary Receipts, etc.)

73 : ETFs

74 : US Closed-End Funds and Unit Investment Units (Depositary Units, Units Of Beneficial Interest, Units Of Limited Partnership Interest, Depositary Receipts, etc.)

75 : Non-US Closed-End Funds and Unit Investment Units (Depositary Units, Units Of Beneficial Interest, Units Of Limited Partnership Interest, Depositary Receipts, etc.)


In [ ]:
remove_indexes = data[(data['SHRCD'] != 10) & (data['SHRCD'] != 11)].index
data.drop(remove_indexes, inplace=True)
remove_indexes = data[(data['EXCHCD'] != 1) & (data['EXCHCD'] != 2) & (data['EXCHCD'] != 3)].index
data.drop(remove_indexes, inplace=True)

data.loc[data["PRC"] < 0, "PRC"] = 0 
data.loc[data["RET"] == 'C', "RET"] = 0
data.loc[data["SHRCD"] == 10, "SHRCD"] = "Ordinary" 
data.loc[data["SHRCD"] == 11, "SHRCD"] = "Common" 
data.loc[data["EXCHCD"] == 1, "EXCHCD"] = "NYSE" 
data.loc[data["EXCHCD"] == 2, "EXCHCD"] = "AMEX" 
data.loc[data["EXCHCD"] == 3, "EXCHCD"] = "NASDAQ" 

In [ ]:
data = data.dropna()
data = data.sort_values('date')
data = data.reset_index()
data = data.drop('index', axis=1)

In [ ]:
lowest_date = datetime.datetime(2010, 1, 29)
remove_indexes = data[data['date'] < lowest_date].index
data.drop(remove_indexes, inplace=True)
data = data.reset_index()
data = data.drop('index', axis=1)

In [ ]:
cd ..

In [ ]:
cd CS 145/CS-145/MarketMaven/data

In [ ]:
filepath = Path('monthly_stock.csv')  
data.to_csv(filepath)  

# FFM Data 

In [ ]:
ffm = pd.read_excel("F-F_Research_Data_Factors.xlsx")

In [ ]:
new_dates = []
for item in ffm.values:
    year = str(item[0])[:4]
    month = str(item[0])[4:]
    new_dates.append(year + "-" + month )

for index in range(len(ffm['Date'])):
    ffm['Date'][index] = new_dates[index]

market_returns = ffm['Mkt-RF'] + ffm['RF']
ffm['Mkt'] = market_returns

monthly_ffm = ffm.loc[:1159]
annual_ffm =  ffm.loc[1163:]

monthly_ffm.Date = pd.to_datetime(monthly_ffm.Date)

for index in range(len(monthly_ffm['Date'])):
    monthly_ffm['Date'][index] = monthly_ffm['Date'][index] + relativedelta(day=31)
#data['RET-Next'] = data.groupby('PERMNO')['RET'].shift(-1)
#data["RET-Next"].fillna(0, inplace=True)'''

In [ ]:
lowest_date = datetime.datetime(2010, 1, 29)
remove_indexes = monthly_ffm[monthly_ffm['Date'] < lowest_date].index
monthly_ffm.drop(remove_indexes, inplace=True)
monthly_ffm = monthly_ffm.reset_index()
monthly_ffm = monthly_ffm.drop('index', axis=1)

In [ ]:
filepath = Path('monthly_stock_ffm.csv')  
monthly_ffm.to_csv(filepath)  

# CRSP STOCK TICKER

Note, we set the permno's ticker to the most recent ticker. 

In [16]:
data = pd.read_csv("current_crsp_tickers.csv")
data = data.dropna()
real_data = pd.read_csv('monthly_stock.csv')

In [ ]:
tickers = data.groupby('PERMNO')['ticker'].apply(set)
ticker_data = tickers.reset_index(name="ticker")
ticker_data['ticker'] = [(list(ticker))[-1] for ticker in ticker_data['ticker']]
ticker_data = ticker_data.set_index('PERMNO')

In [23]:
highest_date = str(datetime.datetime(2020, 12, 31))[:10]
second_highest_date = str(datetime.datetime(2020, 11, 30))[:10]

last_data = real_data[(real_data['date'] != highest_date) & (real_data['date'] != second_highest_date)].index
real_data.drop(last_data, inplace=True)
curr_permnos = list(real_data.permno.unique())

remove_indexes = data[data.permno.isin(curr_permnos) == False].index
data.drop(remove_indexes, inplace=True)
data = data.reset_index()
data = data.drop('index', axis=1)
data = data.set_index('permno')

In [24]:
# now drop all rows from data where the permno is not in the curr_permnos list 
data

,ticker_symbol
permno,
10026,JJSF
10028,MINT
10032,PLXS
10044,RMCF
10051,SEQL
...,...
93422,QEP
93423,SIX
93426,VPG


In [25]:
filepath = Path('current_crsp_tickers.csv')  
data.to_csv(filepath)  